In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

llama_2_docs = SimpleDirectoryReader(input_files=['./assets/llama2.pdf'], filename_as_id=True).load_data()
vector_index = VectorStoreIndex.from_documents(llama_2_docs)

In [2]:
vector_engine = vector_index.as_chat_engine()

In [4]:
from llama_index import ListIndex, ServiceContext
from llama_index import SummaryIndex

# ListIndex -> SummaryIndex 로 바뀜
service_context = ServiceContext.from_defaults(chunk_size=1024)
nodes = service_context.node_parser.get_nodes_from_documents(llama_2_docs)
list_index = ListIndex(nodes)
summary_index = SummaryIndex(nodes)

In [5]:
summary_index

In [6]:
list_index

In [8]:
list_query_engine = list_index.as_chat_engine(
    response_mode = "tree_summarize", use_async=True
)

In [5]:
from llama_index.tools.query_engine import QueryEngineTool

# 구체적인 정보를 검색할 수 있는 vector engine과 요약된 정보로 되어있는 list query engine으로 구성
list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description='Useful for summarization of the paper about llama2'
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_engine,
    description='Useful for retrieving of specific content about llama2'
)

In [6]:
from llama_index.selectors.pydantic_selectors import PydanticSingleSelector
from llama_index.query_engine.router_query_engine import RouterQueryEngine

# 
query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool
    ]
)

In [7]:
from llama_index.response.pprint_utils import pprint_response
response = query_engine.query('What is Llama2?')
pprint_response(response, show_source=True)

Final Response: Llama2 is a query engine tool that allows you to run
natural language queries against a knowledge base and get back natural
language responses. It is designed to make it easier for users to
interact with knowledge bases and retrieve information in a more
conversational manner.
